In [12]:
import numpy as np
import pandas as pd
import math

DATA_PATH = "C:\\Users\\gaoch\\MA\\Badminton weights\\P01_B\\"

SHEET_NAME = ["Segment Position", "Segment Velocity"]

SIMPLIFIED_JOINTS = ["Head", "Neck", "Left Upper Arm", "Right Upper Arm", "Left Forearm", "Right Forearm",
                     "Left Hand", "Right Hand", "Pelvis", "Left Upper Leg", "Right Upper Leg",
                     "Left Lower Leg", "Right Lower Leg", "Left Foot", "Right Foot"]

SKELETON_CONNECTION_MAP = [["Head", "Neck"],
                         ["Neck", "Left Upper Arm"],
                         ["Left Upper Arm", "Left Forearm"],
                         ["Left Forearm", "Left Hand"],
                         ["Neck", "Right Upper Arm"],
                         ["Right Upper Arm", "Right Forearm"],
                         ["Right Forearm", "Right Hand"],
                         ["Neck", "Pelvis"],
                         ["Pelvis", "Left Upper Leg"],
                         ["Left Upper Leg", "Left Lower Leg"],
                         ["Left Lower Leg", "Left Foot"],
                         ["Pelvis", "Right Upper Leg"],
                         ["Right Upper Leg", "Right Lower Leg"],
                         ["Right Lower Leg", "Right Foot"]]

used_cols = []
for joints in SIMPLIFIED_JOINTS:
    used_cols.append(joints+" x")
    used_cols.append(joints+" y")
    used_cols.append(joints+" z")

def read_data(file_name):
    return pd.read_excel(DATA_PATH + file_name,
                         sheet_name = SHEET_NAME,
                         usecols = used_cols)


motion_recording = read_data("P01_B_noweightwithgloves.xlsx")


In [71]:
segment_position = motion_recording["Segment Position"]

def euclidean_distance(point_0, point_1):
    return np.sqrt(sum(np.power(np.subtract(point_0.values, point_1.values),2)))

body_size = {}
for joint_connection in SKELETON_CONNECTION_MAP:
    endpoint_0 = joint_connection[0]
    endpoint_1 = joint_connection[1]
    distance = euclidean_distance(segment_position[[endpoint_0+" x", endpoint_0+" y", endpoint_0+" z"]].iloc[0], segment_position[[endpoint_1+" x", endpoint_1+" y", endpoint_1+" z"]].iloc[0])
    body_size[(endpoint_0, endpoint_1)] = distance

print(body_size)

{('Head', 'Neck'): 0.09022030140538195, ('Neck', 'Left Upper Arm'): 0.19447754755808735, ('Left Upper Arm', 'Left Forearm'): 0.31074836673603157, ('Left Forearm', 'Left Hand'): 0.21463246156034563, ('Neck', 'Right Upper Arm'): 0.1900423735586651, ('Right Upper Arm', 'Right Forearm'): 0.3107369349505133, ('Right Forearm', 'Right Hand'): 0.21462487369865568, ('Neck', 'Pelvis'): 0.4713823327915992, ('Pelvis', 'Left Upper Leg'): 0.10375194613966818, ('Left Upper Leg', 'Left Lower Leg'): 0.5814388004631459, ('Left Lower Leg', 'Left Foot'): 0.4149016305093534, ('Pelvis', 'Right Upper Leg'): 0.10375364252633373, ('Right Upper Leg', 'Right Lower Leg'): 0.5814388813977328, ('Right Lower Leg', 'Right Foot'): 0.4149019387087036}


In [77]:
segment_velocity = motion_recording["Segment Velocity"]

angular_velocity = pd.DataFrame()
for joint_connection in SKELETON_CONNECTION_MAP:
    endpoint_0 = joint_connection[0]
    endpoint_1 = joint_connection[1]
    angular_velocity[(endpoint_1, endpoint_0, "x")] = (segment_velocity[endpoint_1+" x"] - segment_velocity[endpoint_0+" x"])/body_size[(endpoint_0, endpoint_1)]
    angular_velocity[(endpoint_1, endpoint_0, "y")] = (segment_velocity[endpoint_1+" y"] - segment_velocity[endpoint_0+" y"])/body_size[(endpoint_0, endpoint_1)]
    angular_velocity[(endpoint_1, endpoint_0, "z")] = (segment_velocity[endpoint_1+" z"] - segment_velocity[endpoint_0+" z"])/body_size[(endpoint_0, endpoint_1)]

print(angular_velocity.head())



   (Neck, Head, x)  (Neck, Head, y)  (Neck, Head, z)  \
0         0.412819        -0.301263         0.170970   
1         0.414865        -0.259588         0.166362   
2         0.417031        -0.216660         0.161596   
3         0.419279        -0.172898         0.156723   
4         0.421567        -0.128717         0.151798   

   (Left Upper Arm, Neck, x)  (Left Upper Arm, Neck, y)  \
0                  -0.046967                   0.152213   
1                  -0.048466                   0.136035   
2                  -0.050583                   0.119470   
3                  -0.053113                   0.102648   
4                  -0.055850                   0.085696   

   (Left Upper Arm, Neck, z)  (Left Forearm, Left Upper Arm, x)  \
0                   0.148656                           0.007657   
1                   0.149404                           0.004741   
2                   0.150544                           0.001665   
3                   0.151946            

In [81]:
def get_sin_cos(opposite, adjacent):
    hypotenuse = math.sqrt(adjacent ** 2 + opposite ** 2)
    return opposite/hypotenuse, adjacent/hypotenuse

def rotate(frame, sin, cos):
    rotation_matrix = np.array([[cos, sin, 0],[-sin, cos, 0],[0, 0, 1]])
    result_array = []
    for joint in range(0, len(frame), 3):
        joint_vector = np.array([[frame.iloc[joint]], [frame.iloc[joint+1]], [frame.iloc[joint+2]]])
        result_array = np.append(result_array, np.dot(rotation_matrix, joint_vector).T)
    return pd.Series(result_array)

def confront(position, velocity):
    sin_cos = pd.DataFrame()
    sin_cos[["sin","cos"]] = position.apply(lambda x: get_sin_cos(x["Left Upper Arm y"] - x["Right Upper Arm y"], x["Left Upper Arm x"] - x["Right Upper Arm x"]), axis=1, result_type="expand")
    velocity[velocity.columns] = velocity.apply(lambda x: rotate(x, sin_cos["sin"][x.name], sin_cos["cos"][x.name]), axis=1, result_type="expand")
    return velocity

confront(segment_position, angular_velocity)
print(angular_velocity.head())

   (Neck, Head, x)  (Neck, Head, y)  (Neck, Head, z)  \
0        -0.481472        -0.171359         0.170970   
1        -0.448262        -0.196367         0.166362   
2        -0.414110        -0.222193         0.161596   
3        -0.379338        -0.248578         0.156723   
4        -0.344268        -0.275256         0.151798   

   (Left Upper Arm, Neck, x)  (Left Upper Arm, Neck, y)  \
0                   0.152220                  -0.046945   
1                   0.139685                  -0.036643   
2                   0.127186                  -0.025606   
3                   0.114715                  -0.014075   
4                   0.102263                  -0.002293   

   (Left Upper Arm, Neck, z)  (Left Forearm, Left Upper Arm, x)  \
0                   0.148656                           0.239706   
1                   0.149404                           0.237676   
2                   0.150544                           0.235578   
3                   0.151946            